In [3]:
# %load ../font.py
#!/usr/bin/env python

# In[ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import platform

if platform.system() == 'Darwin': #맥
        plt.rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': #윈도우
        plt.rc('font', family='Malgun Gothic') 
elif platform.system() == 'Linux': #리눅스 (구글 콜랩)
        plt.rc('font', family='Malgun Gothic') 

plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결



In [19]:
# 분석할 데이터 load
df = pd.read_csv('dataset/auto-mpg.csv', header=None)

df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name']


df.info()
df.horsepower.head()
# 데이터 전처리
# horsepower를 구간으로 나눠서 분석, object -> float
# could not convert string to float: '?' (?를 찾았음)
# ?를 nan 처리 후 행을 삭제
df.horsepower.replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], axis=0, inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


In [28]:
df.horsepower = df.horsepower.astype('float')
df.info()

# horsepower를 구간으로 나눠서 분석 - 구간을 np.histogram
count, bin_value = np.histogram(df['horsepower'], bins=3)
print(count ,bin_value)

# 구간의 이름을 부여
bin_names = ['저출력', '보통출력', '고출력']

# 분석하고자 하는 자료에 bin_names를 추가
df['hp_bin'] = pd.cut(x=df['horsepower'], # 처리 데이터
                     bins = bin_value,    # 경계값 리스트
                      labels = bin_names, # 구간의 이름 리스트
                      include_lowest='True'
                     )
df[['horsepower','hp_bin']].head(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           392 non-null    float64 
 1   cylinders     392 non-null    int64   
 2   displacement  392 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   weight        392 non-null    float64 
 5   acceleration  392 non-null    float64 
 6   model year    392 non-null    int64   
 7   origin        392 non-null    int64   
 8   name          392 non-null    object  
 9   hp_bin        392 non-null    category
dtypes: category(1), float64(5), int64(3), object(1)
memory usage: 31.1+ KB
[257 103  32] [ 46.         107.33333333 168.66666667 230.        ]


,horsepower,hp_bin
0,130.0,보통출력
1,165.0,보통출력
2,150.0,보통출력
3,150.0,보통출력
4,140.0,보통출력
5,198.0,고출력
6,220.0,고출력
7,215.0,고출력
8,225.0,고출력
9,190.0,고출력


In [61]:
# 분류형 자료를 더미변수로 전환
h_dummy = pd.get_dummies(df['hp_bin'], prefix='hp')
h_dummy
df = pd.concat([df, h_dummy], axis=0)
df.info()


titanic = sns.load_dataset('titanic')
titanic['alive'].unique()
alive_yes =  pd.get_dummies(titanic['alive'],prefix='alive', drop_first=True)

df1 = pd.concat([titanic, alive_yes], axis=1)
df1[['alive', 'alive_yes']].head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 784 entries, 0 to 397
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           392 non-null    float64 
 1   cylinders     392 non-null    float64 
 2   displacement  392 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   weight        392 non-null    float64 
 5   acceleration  392 non-null    float64 
 6   model year    392 non-null    float64 
 7   origin        392 non-null    float64 
 8   name          392 non-null    object  
 9   hp_bin        392 non-null    category
 10  hp_저출력        392 non-null    float64 
 11  hp_보통출력       392 non-null    float64 
 12  hp_고출력        392 non-null    float64 
dtypes: category(1), float64(11), object(1)
memory usage: 80.5+ KB


,alive,alive_yes
0,no,0
1,yes,1
2,yes,1
3,yes,1
4,no,0


In [77]:
# one_hot_encoding 으로 더미 변수 생성
from sklearn import preprocessing

# 전처리를 위한 encoder 객체 생성
label_encoder = preprocessing.LabelEncoder()  # label_encoder 생성
onehot_encoder = preprocessing.OneHotEncoder()# onehot_encoder 생성

# label encoder 문자열 함수를 숫자형 범주로 변환
onehot_label = label_encoder.fit_transform(df['hp_bin'])
print(onehot_label.ndim, onehot_label.dtype, onehot_label.size)

# 2차원 행렬로 변환 
onehot_reshape = onehot_label.reshape(len(onehot_label), 1)
print(onehot_reshape.ndim, onehot_reshape.dtype, onehot_reshape.size)
onehot_reshape[: 10]

# 희소 행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshape)
print(onehot_fitted)

1 int64 784
2 int64 784
  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
  (15, 2)	1.0
  (16, 2)	1.0
  (17, 2)	1.0
  (18, 2)	1.0
  (19, 2)	1.0
  (20, 2)	1.0
  (21, 2)	1.0
  (22, 2)	1.0
  (23, 1)	1.0
  (24, 2)	1.0
  :	:
  (759, 3)	1.0
  (760, 3)	1.0
  (761, 3)	1.0
  (762, 3)	1.0
  (763, 3)	1.0
  (764, 3)	1.0
  (765, 3)	1.0
  (766, 3)	1.0
  (767, 3)	1.0
  (768, 3)	1.0
  (769, 3)	1.0
  (770, 3)	1.0
  (771, 3)	1.0
  (772, 3)	1.0
  (773, 3)	1.0
  (774, 3)	1.0
  (775, 3)	1.0
  (776, 3)	1.0
  (777, 3)	1.0
  (778, 3)	1.0
  (779, 3)	1.0
  (780, 3)	1.0
  (781, 3)	1.0
  (782, 3)	1.0
  (783, 3)	1.0


In [85]:
# 시계열 데이터 처리
df = pd.read_csv('dataset/stock-data.csv')
df.info()

df['new_Date'] = pd.to_datetime(df['Date']) # 데이터 타입이 다름
df.info()

# new_Date를 인덱스로 컬럼으로 설정, Date 컬럼을 삭제

df.set_index(['new_Date'], inplace=True)
df.head()

df.drop('Date', axis=1, inplace=True)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      20 non-null     object        
 1   Close     20 non-null     int64         
 2   Start     20 non-null     int64         
 3   High      20 non-null     int64         
 4   Low       20 non-null     int64         
 5   Volume    20 non-null     int64         
 6   new_Date  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(

,Close,Start,High,Low,Volume
new_Date,,,,,
2018-07-02,10100,10850,10900,10000,137977
2018-06-29,10700,10550,10900,9990,170253
2018-06-28,10400,10900,10950,10150,155769
2018-06-27,10900,10800,11050,10500,133548
2018-06-26,10800,10900,11000,10700,63039


In [86]:
ts_ms = pd.date_range(start='2019-01-01',    # 날짜 범위의 시작
                   end=None,                 # 날짜 범위의 끝
                   periods=6,                # 생성할 Timestamp의 개수
                   freq='MS',                # 시간 간격 (MS: 월의 시작일)
                   tz='Asia/Seoul')          # 시간대(timezone)
print(ts_ms)
print('\n')

# 월 간격, 월의 마지막 날 기준
ts_me = pd.date_range('2019-01-01', periods=6, 
                   freq='M',              # 시간 간격 (M: 월의 마지막 날)
                   tz='Asia/Seoul')       # 시간대(timezone)
print(ts_me)
print('\n')

# 분기(3개월) 간격, 월의 마지막 날 기준
ts_3m = pd.date_range('2019-01-01', periods=6, 
                   freq='3M',             # 시간 간격 (3M: 3개월)
                   tz='Asia/Seoul')       # 시간대(timezone)
print(ts_3m)
# Period 배열 만들기 - 1개월 길이
pr_m = pd.period_range(start='2019-01-01',     # 날짜 범위의 시작
                   end=None,                   # 날짜 범위의 끝
                   periods=3,                  # 생성할 Period 개수
                   freq='M')                   # 기간의 길이 (M: 월)
print(pr_m)
print('\n')

# Period 배열 만들기 - 1시간 길이
pr_h = pd.period_range(start='2019-01-01',     # 날짜 범위의 시작
                   end=None,                   # 날짜 범위의 끝
                   periods=3,                  # 생성할 Period 개수
                   freq='H')                   # 기간의 길이 (H: 시간)
print(pr_h)
print('\n')

# Period 배열 만들기 - 2시간 길이
pr_2h = pd.period_range(start='2019-01-01',    # 날짜 범위의 시작
                   end=None,                   # 날짜 범위의 끝
                   periods=3,                  # 생성할 Period 개수
                   freq='2H')                  # 기간의 길이 (H: 시간)
print(pr_2h)


DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-02-01 00:00:00+09:00',
               '2019-03-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-05-01 00:00:00+09:00', '2019-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-02-28 00:00:00+09:00',
               '2019-03-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-05-31 00:00:00+09:00', '2019-06-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')


DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-07-31 00:00:00+09:00', '2019-10-31 00:00:00+09:00',
               '2020-01-31 00:00:00+09:00', '2020-04-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='3M')
PeriodIndex(['2019-01', '2019-02', '2019-03'], dtype='period[M]')


PeriodIndex(['2019-01-01 00:00', '2019-01-01 01:00', '2019-01-01 02:00'], dtype='perio

In [94]:
# 날짜 데이터를 분리 년 ~ 월 ~ 일 분리
df = pd.read_csv('dataset/stock-data.csv')

df['new_Date'] = pd.to_datetime(df['Date'])

df['Year'] = df['new_Date'].dt.year
df['Month'] = df['new_Date'].dt.month
df['Day'] = df['new_Date'].dt.day
df



,Date,Close,Start,High,Low,Volume,new_Date,Year,Month,Day
0,2018-07-02,10100,10850,10900,10000,137977,2018-07-02,2018,7,2
1,2018-06-29,10700,10550,10900,9990,170253,2018-06-29,2018,6,29
2,2018-06-28,10400,10900,10950,10150,155769,2018-06-28,2018,6,28
3,2018-06-27,10900,10800,11050,10500,133548,2018-06-27,2018,6,27
4,2018-06-26,10800,10900,11000,10700,63039,2018-06-26,2018,6,26
5,2018-06-25,11150,11400,11450,11000,55519,2018-06-25,2018,6,25
6,2018-06-22,11300,11250,11450,10750,134805,2018-06-22,2018,6,22
7,2018-06-21,11200,11350,11750,11200,133002,2018-06-21,2018,6,21
8,2018-06-20,11550,11200,11600,10900,308596,2018-06-20,2018,6,20
9,2018-06-19,11300,11850,11950,11300,180656,2018-06-19,2018,6,19


In [95]:
titanic = sns.load_dataset('titanic')

# age 와 fare 컬럼만 추출
df = titanic.loc[:, ['age', 'fare']]
df.head()

,age,fare
0,22.0,7.2500
1,38.0,71.2833
2,26.0,7.9250
3,35.0,53.1000
4,35.0,8.0500


In [97]:
# 숫자에 10을 더한 값을 실행
def add_age10(n):
    return n + 10

# 두 객체의 합을 구하는
def add_two_obj(a, b):
    return a + b

print(add_age10(10), add_two_obj(10, 20))

20 30


In [102]:
# age 컬럼의 값에 10을 더한 값을 실행
# seriese 에 함수 매핑 -> 시리즈.apply(함수)
df['age+10'] = df['age'].apply(add_age10)
df.head()

# age와 fare의 값을 더함
df['age+fare'] = df['age'].apply(add_two_obj, b=20)
df.head()


df['age'].apply(lambda x: add_age10(x))

0      32.0
1      48.0
2      36.0
3      45.0
4      45.0
       ... 
886    37.0
887    29.0
888     NaN
889    36.0
890    42.0
Name: age, Length: 891, dtype: float64

In [103]:
df.applymap(add_age10) # 데이터프레임 모든 원소에 함수 적용

,age,fare,age+10,age+fare
0,32.0,17.2500,42.0,52.0
1,48.0,81.2833,58.0,68.0
2,36.0,17.9250,46.0,56.0
3,45.0,63.1000,55.0,65.0
4,45.0,18.0500,55.0,65.0
...,...,...,...,...
886,37.0,23.0000,47.0,57.0
887,29.0,40.0000,39.0,49.0
888,NaN,33.4500,NaN,NaN
889,36.0,40.0000,46.0,56.0


In [105]:
# 시리즈 객체에 함수 매핑 : 시리즈.apply(함수)
df.apply(lambda x: x.isnull())
df.apply(lambda x: add_two_obj(x['age'],x['fare']), axis=1)

0       29.2500
1      109.2833
2       33.9250
3       88.1000
4       43.0500
         ...   
886     40.0000
887     49.0000
888         NaN
889     56.0000
890     39.7500
Length: 891, dtype: float64

In [110]:
df.apply(min, axis=1) # 행 단위로 함수 적용
df.apply(min) # axis = 0 , 열 단위로 함수 적용

0       7.250
1      38.000
2       7.925
3      35.000
4       8.050
        ...  
886    13.000
887    19.000
888       NaN
889    26.000
890     7.750
Length: 891, dtype: float64

In [135]:
# 전체 age의 평균, 인원수를 구하세요
avg_age = df['age'].mean()
avg_age
print(avg_age, '\n', len(df))
# auto-mpg.csv 파일을 데이터프레임으로 적용 
mpg_df = sns.load_dataset('mpg')
# mpg-hors = mpg + hors로 컬럼 추가
mpg_df.info()
mpg_df['mpg_hors'] = mpg_df.apply(lambda x : add_two_obj(x['mpg'], x['horsepower']), axis=1)
mpg_df.head(5)
# df['age'].mean()
# len(df['age'])



29.69911764705882 
 891
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model_year    398 non-null    int64  
 7   origin        398 non-null    object 
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,mpg_hors
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,148.0
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,180.0
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,168.0
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,166.0
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,157.0


In [151]:
# 데이터프레임 객체에 함수 매핑
# 데이터프레임.pipe(함수)

def missing_value(x):
    return x.isnull()

def missing_count(x):
    return missing_value(x).sum()

def total_number_missing(x):
    return missing_count(x).sum()



In [154]:
result_value = df.pipe(missing_value) 
print(type(result_value)) # 데이터프레임

result_value = df.pipe(missing_count)
print(type(result_value)) # 시리즈

result_value = df.pipe(total_number_missing)
print(type(result_value)) # 값

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'numpy.int64'>


In [ ]:
# 시리즈의 각원소에 함수를 적용 : sr.apply(함수)
# 데이터프레임의 각 원소에 함수를 적용 : df.applymap(함수)
# 데이터프레임의 행(인덱스)별, 컬럼별 함수 적용
# df.apply(함수, axis=0) axis=0는 컬럼별 적용, axis=1 행별 적용
# 데이터프레임을 인자로 받는 함수 : df.pipe(함수)
# retrun 타입에 따라 데이터 타입이 결정 됨

In [ ]:
# 열 순서 변경 : df[열 이름 리스트]

titanic.head() # survived pclass sex age

df_titanic = titanic.loc[:, 'survived' : 'age']
df_titanic

In [170]:
# column 명을 정렬해서 재배치
print(sorted(list(df_titanic.columns.values)))

df_titanic = df_titanic[sorted(list(df_titanic.columns.values))]
df_titanic

['age', 'pclass', 'sex', 'survived']


,age,pclass,sex,survived
0,22.0,3,male,0
1,38.0,1,female,1
2,26.0,3,female,1
3,35.0,1,female,1
4,35.0,3,male,0
...,...,...,...,...
886,27.0,2,male,0
887,19.0,1,female,1
888,NaN,3,female,0
889,26.0,1,male,1


In [165]:
# column 명을 역으로 정렬해서 재배치
print(sorted(list(df_titanic.columns.values)))

df_titanic = df_titanic[reversed(df_titanic.columns.values)]
df_titanic

['age', 'pclass', 'sex', 'survived']


,survived,sex,pclass,age
0,0,male,3,22.0
1,1,female,1,38.0
2,1,female,3,26.0
3,1,female,1,35.0
4,0,male,3,35.0
...,...,...,...,...
886,0,male,2,27.0
887,1,female,1,19.0
888,0,female,3,NaN
889,1,male,1,26.0


In [203]:
# 열 분리
stock_df = pd.read_excel('dataset/stock_data.xlsx')
stock_df

# 년, 월, 일을 분리 리스트로 [2018, 07, 02]
# stock_df.연월일 데이터 타입 확인 - 
print(stock_df['연월일'].dtype)

# datetime 으로 되어 있음 -> str
# 연, 월, 일로 분리
stock_df['연월일'] = stock_df['연월일'].astype('str')
print(stock_df['연월일'].dtype)
dates = stock_df['연월일'].str.split('-')

stock_df['연'] = dates.str.get(0)
stock_df['월'] = dates.str.get(1)
stock_df['일'] = dates.str.get(2)

stock_df.head()

datetime64[ns]
object


,연월일,당일종가,전일종가,시가,고가,저가,거래량,연,월,일
0,2018-07-02,10100,600,10850,10900,10000,137977,2018,07,02
1,2018-06-29,10700,300,10550,10900,9990,170253,2018,06,29
2,2018-06-28,10400,500,10900,10950,10150,155769,2018,06,28
3,2018-06-27,10900,100,10800,11050,10500,133548,2018,06,27
4,2018-06-26,10800,350,10900,11000,10700,63039,2018,06,26


In [213]:
# stock-data.csv 파일을 읽어서 연,월,일 구분해서 컬럼 추가

stock_df1 = pd.read_csv('dataset/stock-data.csv')

stock_df1['Date'] = stock_df1['Date']
dates1 = stock_df1['Date'].str.split('-')


stock_df1['연'] = dates.str.get(0)
stock_df1['월'] = dates.str.get(1)
stock_df1['일'] = dates.str.get(2)
stock_df1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB


,Date,Close,Start,High,Low,Volume,연,월,일
0,2018-07-02,10100,10850,10900,10000,137977,2018,07,02
1,2018-06-29,10700,10550,10900,9990,170253,2018,06,29
2,2018-06-28,10400,10900,10950,10150,155769,2018,06,28
3,2018-06-27,10900,10800,11050,10500,133548,2018,06,27
4,2018-06-26,10800,10900,11000,10700,63039,2018,06,26


In [235]:
titanic

mask1 = titanic.loc[(10 <= titanic['age']) & (titanic['sex'] == 'female')]

age_60 = titanic.loc[(titanic['age'] >= 60) & (titanic['alive'] == 'yes'),
                    ['age', 'sex', 'class', 'alive']]  
age_60


,age,sex,class,alive
275,63.0,female,First,yes
366,60.0,female,First,yes
483,63.0,female,Third,yes
570,62.0,male,Second,yes
587,60.0,male,First,yes
630,80.0,male,First,yes
829,62.0,female,First,yes


In [237]:
# sibsp 컬럼 : 형제 또는 배우자의 수  -> 수가 3,4,5인 승객의
# age, class, sibsp, alive 컬럼만 축출
mask3 = titanic['sibsp'] == 3
mask4 = titanic['sibsp'] == 4
mask5 = titanic['sibsp'] == 5

col_list = ['age', 'class', 'sibsp', 'alive']

titanic_df = titanic.loc[ mask3 | mask4 | mask5, col_list]
titanic_df


,age,class,sibsp,alive
7,2.0,Third,3,no
16,2.0,Third,4,no
24,8.0,Third,3,no
27,19.0,First,3,no
50,7.0,Third,4,no
59,11.0,Third,5,no
63,4.0,Third,3,no
68,17.0,Third,4,yes
71,16.0,Third,5,no
85,33.0,Third,3,yes


In [242]:
# isin() 함수를 이용하여 추출
isin_filter = titanic['sibsp'].isin([3,4,5])
isin_titanic = titanic.loc[isin_filter, col_list]

isin_titanic

,age,class,sibsp,alive
7,2.0,Third,3,no
16,2.0,Third,4,no
24,8.0,Third,3,no
27,19.0,First,3,no
50,7.0,Third,4,no
59,11.0,Third,5,no
63,4.0,Third,3,no
68,17.0,Third,4,yes
71,16.0,Third,5,no
85,33.0,Third,3,yes


In [262]:

# titanic 데이터를 로드해서
df = sns.load_dataset('titanic')
df
# Nan 데이터를 처리 : age는 평균값으로, deck 컬럼은 삭제,
df['age'].fillna(df['age'].mean(axis=0), inplace=True)
df.dropna(axis=1, thresh=300, inplace=True)
df.info()

df[['embarked','embark_town']].fillna(method='ffill', inplace=True)
df.info()

# 다른 컬럼은 이전 값을 대체
# 평균 age보다 적은 age의 승객 중에서 여성과 생존자 만 
# age, sex, alive, class 컬럼만 추출
col_list = ['age', 'sex', 'alive', 'class']
mask1 = (df['age'] < df['age'].mean()) & (df.sex =='female')\
     & (df['alive'] =='yes')
df_1 = df.loc[ mask1, col_list]
df_1

# 가족의 수가 4,5 명인 승객의 나이, 가족수, 생존여부를 추출
df.loc[ df['sibsp'].isin([4,5]), [ 'age','sibsp','alive']]


df = sns.load_dataset('titanic')
df.info()

df.dropna(inplace=True, axis=1,thresh=300)

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [267]:
# 데이터프레임 합치기 : pd.concat(데이터프레임 리스트)
df1 = pd.DataFrame({'a': ['a0', 'a1', 'a2', 'a3'],
                    'b': ['b0', 'b1', 'b2', 'b3'],
                    'c': ['c0', 'c1', 'c2', 'c3']},
                    index=[0, 1, 2, 3])
 
df2 = pd.DataFrame({'a': ['a2', 'a3', 'a4', 'a5'],
                    'b': ['b2', 'b3', 'b4', 'b5'],
                    'c': ['c2', 'c3', 'c4', 'c5'],
                    'd': ['d2', 'd3', 'd4', 'd5']},
                    index=[2, 3, 4, 5])

result1 = pd.concat([df1, df2])
print(result1)

# 기존의 인덱스를 무시하고 인덱스 재설정
result2 = pd.concat([df1, df2], ignore_index=True)
print(result2)


# 열 단위로 합성
result3 = pd.concat([df1, df2], axis=1) # 인덱스가 없으면 Nan
print(result3)


# 인덱스가 같은 행만 조인
result4 = pd.concat([df1, df2], join='inner') # 인덱스가 없으면 Nan
print(result4)

    a   b   c    d
0  a0  b0  c0  NaN
1  a1  b1  c1  NaN
2  a2  b2  c2  NaN
3  a3  b3  c3  NaN
2  a2  b2  c2   d2
3  a3  b3  c3   d3
4  a4  b4  c4   d4
5  a5  b5  c5   d5
    a   b   c    d
0  a0  b0  c0  NaN
1  a1  b1  c1  NaN
2  a2  b2  c2  NaN
3  a3  b3  c3  NaN
4  a2  b2  c2   d2
5  a3  b3  c3   d3
6  a4  b4  c4   d4
7  a5  b5  c5   d5
     a    b    c    a    b    c    d
0   a0   b0   c0  NaN  NaN  NaN  NaN
1   a1   b1   c1  NaN  NaN  NaN  NaN
2   a2   b2   c2   a2   b2   c2   d2
3   a3   b3   c3   a3   b3   c3   d3
4  NaN  NaN  NaN   a4   b4   c4   d4
5  NaN  NaN  NaN   a5   b5   c5   d5
    a   b   c
0  a0  b0  c0
1  a1  b1  c1
2  a2  b2  c2
3  a3  b3  c3
2  a2  b2  c2
3  a3  b3  c3
4  a4  b4  c4
5  a5  b5  c5


In [270]:
sr1 = pd.Series(['e0','e1','e2','e3',], name='e')
sr2 = pd.Series(['f0','f1','f2'], name='f', index=[3,4,5])
sr3 = pd.Series(['g0','g1','g2','g3',], name='g')

In [271]:
# df1 에 sr1를 추가
res = pd.concat([df1, sr1], axis=1)
res

,a,b,c,e
0,a0,b0,c0,e0
1,a1,b1,c1,e1
2,a2,b2,c2,e2
3,a3,b3,c3,e3


In [273]:
# df2 에 sr2를 추가
res1 = pd.concat([df2, sr2], axis=1, sort=True)
res1

,a,b,c,d,f
2,a2,b2,c2,d2,NaN
3,a3,b3,c3,d3,f0
4,a4,b4,c4,d4,f1
5,a5,b5,c5,d5,f2


In [275]:
# sr1 과 sr3를 결합
res2 = pd.concat([sr1, sr3], axis=1)
res2

,e,g
0,e0,g0
1,e1,g1
2,e2,g2
3,e3,g3


In [277]:
# 데이터프레임 병합 : merge()

df1 = pd.read_excel('dataset/stock_price.xlsx')
df2 = pd.read_excel('dataset/stock_valuation.xlsx')


In [281]:
merge_inner = pd.merge(df1, df2)
merge_inner

,id,stock_name,value,price,name,eps,bps,per,pbr
0,130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068,15.695091,1.829178
1,139480,이마트,239230.833333,254500,이마트,18268.166667,295780,13.931338,0.860437
2,145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090,14.283226,0.758627
3,185750,종근당,40293.666667,100500,종근당,3990.333333,40684,25.185866,2.470259
4,204210,모두투어리츠,3093.333333,3475,모두투어리츠,85.166667,5335,40.802348,0.651359


In [282]:
merge_outer = pd.merge(df1, df2, how='outer', on='id')
merge_outer

,id,stock_name,value,price,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000.0,NaN,NaN,NaN,NaN,NaN
1,130960,CJ E&M,58540.666667,98900.0,CJ E&M,6301.333333,54068.0,15.695091,1.829178
2,138250,엔에스쇼핑,14558.666667,13200.0,NaN,NaN,NaN,NaN,NaN
3,139480,이마트,239230.833333,254500.0,이마트,18268.166667,295780.0,13.931338,0.860437
4,142280,녹십자엠에스,468.833333,10200.0,NaN,NaN,NaN,NaN,NaN
5,145990,삼양사,82750.000000,82000.0,삼양사,5741.000000,108090.0,14.283226,0.758627
6,185750,종근당,40293.666667,100500.0,종근당,3990.333333,40684.0,25.185866,2.470259
7,192400,쿠쿠홀딩스,179204.666667,177500.0,NaN,NaN,NaN,NaN,NaN
8,199800,툴젠,-2514.333333,115400.0,NaN,NaN,NaN,NaN,NaN
9,204210,모두투어리츠,3093.333333,3475.0,모두투어리츠,85.166667,5335.0,40.802348,0.651359
